### Create a DLT Pipeline using SQL

### Create your First DLT pipeline
- Observe the code in this notebook
- Click on Jobs & Pipelines: and create a DLT pipeline (ETL Pipeline) and provide the following in formation:
  - Pipeline name: `<Enter unique name>`
  - Product edition: **Advance**
  - Pipeline mode: **Triggered**
  - Paths: `<Navigate to this notebook and select this notebook>`.
  - Storage option: **hive_metastore**
  - Storage location: **dbfs:/pipelines/tables**
  - Default schema: **default**
  - Cluster mode: **Fixed size**
  - Workers: **0**
  - Driver type: **Ds3v2**
  - Click Create
  - On the DLT pipeline page, click Start.
  - Navigate to **Catalog Explorer** after you pipeline start
    - Under hive_metastore look to **top_n** table

In [0]:
-- USE CATALOG hive_metastore;
-- DROP SCHEMA IF EXISTS dltexample CASCADE;
-- CREATE SCHEMA dltexample;

In [0]:
-- Bronze layer: Raw data ingestion
CREATE OR REFRESH STREAMING TABLE taxi_raw_records 
(CONSTRAINT valid_distance EXPECT (trip_distance > 0.0) ON VIOLATION DROP ROW)
AS SELECT *
FROM STREAM(samples.nyctaxi.trips);

-- Silver layer 1: Flagged rides
CREATE OR REFRESH STREAMING TABLE flagged_rides 
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  pickup_zip as zip, 
  fare_amount, trip_distance
FROM
  STREAM(LIVE.taxi_raw_records)
WHERE ((pickup_zip = dropoff_zip AND fare_amount > 50) OR
       (trip_distance < 5 AND fare_amount > 50));

-- Silver layer 2: Weekly statistics
CREATE OR REFRESH MATERIALIZED VIEW weekly_stats
AS SELECT
  date_trunc("week", tpep_pickup_datetime) as week,
  AVG(fare_amount) as avg_amount,
  AVG(trip_distance) as avg_distance
FROM
 live.taxi_raw_records
GROUP BY week
ORDER BY week ASC;

-- Gold layer: Top N rides to investigate
CREATE OR REPLACE MATERIALIZED VIEW top_n
AS SELECT
  weekly_stats.week,
  ROUND(avg_amount,2) as avg_amount, 
  ROUND(avg_distance,3) as avg_distance,
  fare_amount, trip_distance, zip 
FROM live.flagged_rides
LEFT JOIN live.weekly_stats ON weekly_stats.week = flagged_rides.week
ORDER BY fare_amount DESC
LIMIT 3;